## UGCA Group Assignment -1 

# Group Members
1. Chetna Singhal (cs57926)
2. David Kinman
3. Prajval Gupta
4. Subhayu Chakravarty
5. Whitt Hyde

### Part 1 & 2 - Script to scrape data from the <a href="https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans">Edmunds.com - Entry level luxury cars forum</a>


In [10]:
python -m pip install -U selenium

SyntaxError: invalid syntax (<ipython-input-10-64be7289b5a0>, line 1)

In [9]:


#Import Statements
import selenium
from selenium import webdriver
import pandas as pd
from time import sleep
from string import punctuation
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
import copy
import matplotlib.pyplot as plt
import numpy as np
from itertools import groupby
from collections import Counter
from nltk import ngrams, FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, webtext
from sklearn import manifold
%pylab inline

ModuleNotFoundError: No module named 'selenium'

In [2]:
## Script to scarpe the data from Edmunds - Entry Level Luxury Cars Forum
driver = webdriver.Chrome(executable_path='chromedriver')

comments = pd.DataFrame(columns = ['Date','user_id','comments']) 

for i in range(723,550,-1):
    driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p'+ str(i))

    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    for x in comment_ids:
        #Extract dates from for each user on a page
        user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')
        
        #Extract user ids from each user on a page
        userid_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text
        
        #Extract Message for each user on a page
        user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]/div/div[1]')[0]
        comment = user_message.text
        
        #Removing block quotes
        total_msg = driver.find_element_by_xpath('//*[@id="' + x + '"]/div/div[3]/div/div[1]')
        block_msg = total_msg.find_elements_by_class_name('UserQuote')
        if block_msg:
            comment = comment.replace(block_msg[0].text,"")
            
        #Adding date, userid and comment for each user in a dataframe    
        comments.loc[len(comments)] = [date,userid,comment]

print(comments.shape)
comments.to_csv("posts.csv", encoding='utf-8')

NameError: name 'webdriver' is not defined

### Part 3 - Data Cleansing

In [3]:
# Remove all punctuation characters from each post and convert text to lowercase for efficient frequency counting
# punctuation includes !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

def remove_punctuations(item):
    for p in punctuation:
        item = item.strip().replace(p,'')
    return item

# Replace car models with brand names for effective analysis - Part B
def model_to_brand(s):
    for i in models.index.values:
        s = s.replace(models["Model"][i].lower(),models["Brand"][i].lower())
    return s

# Function to remove stopwords
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 

# Function to check if a brand is mentioned in a post or not
def brand_count(item):
    if brand in item:
        return 1
    else:
        return 0
    
def model_to_capbrand(s):
    for i in models.index.values:
        if models["Brand"][i].lower() in s:
            s = s.replace(models["Brand"][i].lower(),models["Brand"][i].upper())
            continue
    return s

def brand_attri_count(brandList, df_a, attributeList):
    comb_count=[]
    car_count={}
    attrib_count={}
    for car in brandList:
        num_car=0
        for comment in df_a.Posts:
                if car in comment:
                    num_car+=1
        car_count[car]=num_car
        for attrib in attributeList:
            num_combined=0
            num_attrib=0
            for comment in df_a.Posts:
                if car in comment and attrib in comment:
                    num_combined+=1
                if attrib in comment:
                    num_attrib+=1
            attrib_count[attrib]=num_attrib
            comb_count.append(((car,attrib),num_combined))
    return [[car_count,attrib_count,comb_count]]

def calculate_lift(car, attrib, combined,attrib_count,car_count):
    if(attrib_count[attrib]==0):
        print(attrib)
    if(car_count[car]==0):
        print(car)
    lift = (5164*combined)/(car_count[car]*attrib_count[attrib])
    return lift

In [4]:
# Load user comments into a dataframe
df = pd.read_csv("posts.csv", usecols=["comments"])
df.columns = ["Posts"]

# Remove newline from each post & drop rows with null values
df = df.replace('\n','', regex=True)
df = df.dropna()

df["Post Length"]= df["Posts"].str.len() 

df["Posts"] = df['Posts'].apply(remove_punctuations)
df['Posts'] = df["Posts"].apply(lambda x: x.lower())
df["Post Length"]= df["Posts"].str.len() 
df.head()

NameError: name 'pd' is not defined

In [5]:
# Replace car models with brand names for effective analysis - Part A
models = pd.read_csv("models.csv", header = 0, names = ['Brand','Model'],encoding='windows-1252')
models['Brand'] = models['Brand'].str.strip(punctuation)
df['Posts'] = df['Posts'].apply(model_to_brand)
df["Post Length"]= df["Posts"].str.len() 

#Download stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Replace the typing errors and misspelled brand names
df['Posts'].replace({'beaudier': 'AUDI', 'preaudiy':'AUDI', 'geaudiing':'AUDI', 'audi0':'AUDI', 'maaudier':'AUDI'}, inplace=True, regex=True)
df['Posts'].replace({'acurax': 'ACURA', 'litacurae':'ACURA', 'exacacuray':'ACURA', 'acuras':'ACURA', 'recenacuray':'ACURA', 'mosacuray':'ACURA', 'apparenacuray':'ACURA', '’':''}, inplace=True, regex=True)
df['Posts'].replace({'thcadillac': 'CADILLAC', 'secadillac':'CADILLAC'}, inplace=True, regex=True)
df['Posts'].replace({'beaudier': 'BMW','bmwi':'BMW'}, inplace=True, regex=True)
df['Posts'].replace({'mercedess': 'MERCEDES','mercedesss': 'MERCEDES', 'rememercedeser':'MERCEDES', 'numercedeser':'MERCEDES', 'numercedesers':'MERCEDES'}, inplace=True, regex=True)
df['Posts'].replace({'nissanion': 'NISSAN'}, inplace=True, regex=True)

# Tokenize the posts
df['Tokens'] = df['Posts'].apply(word_tokenize).apply(set).apply(list)
df['Tokens'] = df['Tokens'].apply(remove_stopwords)
df.head()

NameError: name 'pd' is not defined

### Count word frequency

In [6]:
count = []
# Count the frequency of the words
for i in range(len(df["Tokens"])):
    count += df.iloc[i]['Tokens']
count = [x.lower() for x in count]
word_freq = nltk.FreqDist(count)

# 500 most frequent words
top_words = word_freq.most_common(500)
top_words

NameError: name 'df' is not defined

## Task A
### Top 10 brands with the frequencies


After the analysis the top 10 brands we found with their frequency counts are:<br>
1. Audi - 2008
2. BMW - 1512
3. Acura - 1376
4. Cadillac - 808
5. Mercedes - 578
6. Honda - 351
7. Ford - 241
8. Vilkswagen - 228
9. Hyundai - 195
10. Infiniti - 189

In [7]:
top_brands_count = []

models_unique = models['Brand'].drop_duplicates().tolist()

for (key, items) in top_words:
    if key in models_unique:
        model_count = (key,items)
        top_brands_count.append(model_count)  

print ('Top 10 brands along with their frequencies are: \n' , top_brands_count[:10])

NameError: name 'models' is not defined

### Calculate Lift Ratios for Top 10 brands 

In [8]:
# Create a new dataframe to store occurence of each brand in top 10 list in each post
top_brands =[]
for brand, count in top_brands_count[:10]:
    top_brands.append(brand) 

brand_df = pd.DataFrame(columns = top_brands)

for brand in top_brands:
    brand_df[brand] = df['Tokens'].apply(brand_count)        

lift_df = pd.DataFrame(columns = top_brands)

for i in range(len(top_brands)):
    new_list = []
    for j in range(len(top_brands)):
        if (i!=j):
            num = ((brand_df[top_brands[i]] + brand_df[top_brands[j]]) > 1).sum()
            dem = brand_df[top_brands[j]].sum()*brand_df[top_brands[i]].sum()
            lift = num * len(brand_df) / dem
            lift_df.loc[top_brands[i],top_brands[j]] = lift

print ('Lift Matrix:\n')
lift_df

NameError: name 'pd' is not defined

### Inverse Lift Matrix

In [ ]:
# Calculating the dissimilarity matrix which is the input for plotting MDS plot

lift_inverse_df = 1/lift_df
np.fill_diagonal(lift_inverse_df.values, 0)

lift_inverse_df

### Multi-Dimensional Scaling (MDS) map

In [ ]:
seed = np.random.RandomState(seed=3)

mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, random_state=seed,
                   dissimilarity="precomputed", n_jobs=1)
pos = mds.fit(lift_inverse_df)
coords = pos.embedding_


plt.subplots_adjust(bottom = 0.1)
plt.scatter(
    coords[:, 0], coords[:, 1], marker = 'o'
    )
for label, x, y in zip(top_brands, coords[:, 0], coords[:, 1]):
    
    plt.annotate(
        label,
        xy = (x, y), xytext = (-5, 5),
        textcoords = 'offset points', ha = 'right', va = 'bottom')

plt.show()

# Task B: 

What insights can you offer brand managers from your analysis in Task A (choose two brands that you can offer the most interesting/useful insights for)? 



### I. Ford
In our lift analysis we found two very strong trends regarding the American auto company based in Detroit: 
- They are mentioned most with foreign (non-American) brands
- Hyundai presents the biggest competition for Ford (2.52 Lift Value) followed closely by Volkswagen (2.20 Lift Value).
#### Foreign Brands
First, Ford finds itself in an interesting position as the American company that prides itself on being America’s leading producers of automobiles finds its brand most closely related to foreign companies. This could mean one of two things: either Ford’s campaign to establish itself as “America’s car manufacturer” is not working, or the push for an “American” image is inconsequential to the consumer when considering the vehicle.
##### American Image
Let’s take the first scenario into account. If it is indeed the case that Ford’s marketing campaigns are ineffective at portraying the brand as domestic, then I would advise brand managers to adjust advertising to make a stronger push to the patriotic appeal. Then, after adjusting the marketing strategies you could run the lift analysis once more to see if Ford’s lift values are more aligned with other domestic brands as well as comparing overall sales figures to make. 
##### Other Explanation
However, it is important to consider the context of the study (Entry Level Luxury Sedans). Ford has always been a leader in the pickup truck industry, and pickup trucks are a very American product, thus it is important to truck consumers that their truck is “American”. But perhaps it is more likely that the consumer of a luxury sedan does not care as much about the vehicle being “American Made”, but rather consider other features (such as safety for instance) as more important. The answer to this question relies in the competition for Ford.
#### Hyundai and Volkswagen Competition
It is likely that this high relation and lift with these foreign brands is due to another feature. When we look to the closest brands in lift we see Ford lines up against Hyundai and Volkswagen. The sedans for those two brands - The “Hyundai Elantra” and “Volkswagen Jetta” are known for their reliability for a reasonable price. This makes sense for Ford historically, as they have been leaders on durability and longevity in their vehicles. With this in mind, there are two steps strategically that I can see success moving forward: become the sole leader in reliability, or add another key feature of their vehicle. 
### Recommendation to Ford’s Brand Manager

    Ford should take the following steps:
    1. Make a stronger pursuit in establishing itself in its competitive positioning as “The American” Brand. They can do this through advertising campaigns on the national level, while appealing to luxury car consumers emotional patriotic side. Luxury consumers are inherently by definition willing to spend a little more. Ford should first do their own analysis to make sure this is in fact something their customer cares about, then continue to highlight its American features.
    2. Ford should expand its core competencies. Ford has always been known as a very reliable brand and that’s a great thing. However I believe it would do Ford to expand upon this and enter a more niche speciality. For instance, they could become the world’s most reliable “green car”, or most reliable “comfort ride.” This is important because right now they stand in line with their competition Hyundai and Volkswagen. If they cannot do something to differentiate in brand, they will likely lose customers to the other two if they were to offer a slightly lower price. 
    
### II. Mercedes

After our lift analysis we found that Mercedes entry level sedans are on an island with the Japanese Infiniti brand with a lift value of 4.34. This is very alarming for Mercedes as traditionally their competition is with higher luxury brands such as Audi and BMW. It is possible this lift value is high for two reasons:  
- Mercedes and Infiniti are both known as unreliable relative to other luxury brands 
- The competitive landscape between the two is close, and this should raise alarms for Mercedes
#### Reliability Issue
One explanation to the strong connection between Infiniti and Mercedes is the reliability problem that has plagued both brands. Consumer Reports marked the Infiniti Q50 as one of the least reliable luxury cars on the market back in 2014. They reported that it was 127% less reliable than the average. In 2014, Consumer Reports also reported that the Mercedes CLA was not only the most unreliable Mercedes in the Auto Reliability Survey, but 140% worse than the average car. So, in other words both brands have a big issue on their hand when it comes to public opinion and reliability
(References: https://www.osv.ltd.uk/are-mercedes-benz-reliable/ , and https://www.osv.ltd.uk/are-infiniti-reliable/ )
#### Mercedes Competition with Infiniti
Mercedes and Infiniti’s core market is luxury vehicles and as such they will share a lot of the same battlegrounds. In fact, Infiniti’s Q Models, and Mercedes-Benz’s C-Class are direct competitors in the entry level luxury sedan field that share a lot of similarities. However there are slight differences in the models that highlight the overall sentiments of the brands. Mercedes’ C-Class delivers more power, it also has a more safety feature, and it even gives a more economic fuel efficient ride. 
(References: https://www.autotrader.com/car-reviews/2017-infiniti-q60-vs-2017-mercedes-benz-c-class-coupe-which-better-258411 )
###### So why are these brands so closely related then???
This golden question is one that needs to be answered fast by the brand managers at Mercedes. In my opinion, the brand needs to realign itself to the market of upper-luxury and stick to its core competency. There are many different levels of C-Class Mercedes, wherein the more you spend -> the better performing model you get. This is done to be competitive at all the levels of the market. Take for instance the Mercedes C300 model, it is the cheapest in the C-Class, and it is made this way so Mercedes can sell to the lower end of the luxury model. 
### Recommendation for Mercedes Brand Manager

    The brand manager needs to distance themself from Infiniti. In order to do this, they must accomplish a few things:
    1. First, they must revamp marketing to show the superior features that one gets when they choose Mercedes. 
    2. Second, they must work to remove the stigma of Mercedes’ reliability (or lack thereof). They do not need to become leaders in reliability, however they must raise the bar from where it stands today. 
    3. Third, I recommend they pull out of the lower-entry level of luxury. While selling the C300 model allows them to reach another customer base, it is opening up the opportunities to compare them to other brands in that market (namely Infiniti). The Infiniti Q60 is no where close to the car that the C43 Mercedes is, but they aren’t close in price either. However, it is clear based on the high lift of 2.39 the two brands are being compared and talked about in union. I believe this is a byproduct of Mercedes entering this market with the C300. I would recommend Mercedes pull the C300, and focus on the market of car that has made them famous.

## Task C

### Attribute - Brand Association

#### The 5 attributes we choose after analysis are 'luxury','performance','space','price','safety'

In [ ]:
df_a=df.copy()
df_a=df_a.drop(['Tokens'], axis=1)

performwords={'powerful':'performance','power':'performance',
              'mileage':'performance','speed':'performance',
              'fast':'performance','transmission':'performance',
              'engine':'performance','suspension':'performance','sport':'performance',
             'high performance':'performance',
              'quick':'performance',
              'good horsepower':'performance',
              'exotic':'performance',
              'excellent':'performance',
              'fuel efficient':'performance',
              'high fuel economy':'performance',
              'high mpg':'performance',
              'good mpg':'performance',
              'good fuel economy':'performance',
              'responsive':'performance',
              'tough':'performance',
              'reliable':'performance'}
#use style 
stylewords={'look':'style','looks':'style','stylish':'style',
            'gorgeous':'style','classy':'style','beauty':'style',
            'good looking':'style',
            'modern':'style','sleek':'style'}


#use luxury
luxurywords={'highend':'luxury','high end':'luxury',
             'luxurious':'luxury','expensive':'luxury',
             'fancy':'luxury','fancier':'luxury',
             'grand':'luxury', 'high-end':'style'}
#Use comfort
comfortwords={'comfortable':'comfort','convenience':'comfort',
              'convenient':'comfort','comfy':'comfort',
              'cozy':'comfort','good cup holders':'comfort','spacious cup holders':'comfort',
              'good cup-holders':'comfort','spacious cup-holders':'comfort'}
#use price
costwords={'cheap':'price','prices':'price',
           'pricing':'price','economic':'price',
           'costeffective':'price','lease':'price',
           'rent':'price','money':'price',
           'good price':'price','good pricing':'price',
           'good listing':'price','sale':'price',
           'economical':'price','affordable':'price',
           'cost-effective':'price','cost effective':'price'
          
          }
#usespace
bigwords={'big':'space','wide':'space','wider':'space',
          'bigger':'space','larger':'space','spacious':'space',
          'huge':'space','legspace':'space',
          'bigger':'space','large':'space',
          'leg-space':'space','leg room':'space',
          'open':'space'        
         }
#use safety
safewords={'safe':'safety','safer':'safety','tested':'safety',
           'test':'safety','crash-test':'safety','crash test':'safety',
           'safety score':'safety','reliable':'safety',
           'reliability':'safety'
          
          }

df_a['Posts'].replace(performwords, inplace=True, regex=True)
df_a['Posts'].replace(stylewords, inplace=True, regex=True)
df_a['Posts'].replace(luxurywords, inplace=True, regex=True)
df_a['Posts'].replace(costwords, inplace=True, regex=True)
df_a['Posts'].replace(bigwords, inplace=True, regex=True)
df_a['Posts'].replace(safewords, inplace=True, regex=True)
df_a['Posts'].replace(comfortwords, inplace=True, regex=True)
df_a['Posts'].replace({"MERCEDESs":"MERCEDES"}, inplace=True, regex=True)
# df_a['Posts'].replace(influencewords, inplace=True, regex=True)

df_a['Posts'] = df_a['Posts'].apply(model_to_capbrand)

df_a['Tokens'] = df_a['Posts'].apply(word_tokenize)
stop_words.update(['thing','anything','much','something','My','We','my','we','year','day','get','dealer','way','would','like','car','cars','time','way','dealer','lot','point','course','im','dont','doesnt','drive','model','brand'])
df_a['Tokens'] = df_a['Tokens'].apply(remove_stopwords)
df_a.head()

In [ ]:
# Apply POS tagging
tagList=[]
for com in df_a['Tokens']:
    tagged = nltk.pos_tag(com) 
    tagList.append(tagged)
print("The POS tagging results are : %s" % format(tagList[0]))

# Flattening into a bag of words
wordSoup = [y for x in tagList for y in x]

# Getting most common Proper nouns and common/abstract nouns
nounList = [word for word in wordSoup if word[1]=='NNP']
adjectiveList = [word for word in wordSoup if (word[1]=='NN')]
nouncountList = Counter(nounList)
print("The most common Proper Nouns are : %s" % format(nouncountList.most_common(5)))
print("------------")
# Most common attributes
adjcountList = Counter(adjectiveList)
print("The most common adjectives are : %s" % format(adjcountList.most_common(5)))
print("------------")

brandList = ['ACURA','AUDI','BMW','CADILLAC','MERCEDES']
attributeList = ['luxury','performance','space','price','safety']

# Lift scores for top 5 brands
lift_scores={}
for i in brand_attri_count(brandList, df_a, attributeList):
    for k,num in i[2]:
        lift_scores[(k[0],k[1])] = calculate_lift(k[0],k[1], num,i[1],i[0])
sorted_lifts = sorted(lift_scores.items(), key=lambda kv: kv[1], reverse=True)
print("The sorted Lift scores for top 5 brands are : %s" % sorted_lifts)

In [ ]:
# Plotting dicts for the MDS map
plotarr={}
for b in brandList:
    plotarr[b]=[]
for i in lift_scores:
    plotarr[i[0]].append(lift_scores[i])

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
colors = ("red", "green", "blue","black","orange")
groups = plotarr.keys()

for lift, color, group in zip(plotarr, colors, groups):
#     plt.scatter(attributeList,plotarr[lift])
#     ax.scatter(attributeList, plotarr[lift])
    ax.scatter(attributeList, plotarr[lift], alpha=0.8, c=color, edgecolors='none', label=group)

plt.xlabel('Combinations')
plt.ylabel('Lift Score')
plt.legend(loc=1)
plt.show()

## Task D 

All of these recommendations are based on the lift value calculations above. The top 5 brands that were mentioned by users within the Edmunds post is Acura, Audi, BMW, Cadillac, and Mercedes. Some notable brands missing are Ford, Toyota, Nissan, Infiniti. Also, we had eight groups of word associations. The eight-word associations are luxury, performance, style, price, space, influence, aspirational, and comfort. We calculated mentions that included all of the models that are part of a brand’s portfolio. For example, if “m3” is mentioned, we equate “m3” with “BMW.” Also, we didn’t have a lot of variance in our lift values. The max and min of the Lift values from Task C were 2.107 and 1.3599. The lack of variance in our lift values between the top 5 brands and the top 5-word associations was somewhat surprising. We expected to have a maximum lift value of 4 and a minimum lift value of 0.2.
 
For nearly all the advertising manager and product manager recommendations, we followed these guidelines. In general, if a lift value for a brand and an attribute is higher than the average lift value, we recommend marketing managers to portray/associate their brand with those attributes. If a lift value for a brand and an attribute is lower than average, we urge product managers to improve the aspect of their product that is associated with that attribute.


### Mercedes Benz 

#### Product Manager

Luckily for the Product Manager, Mercedes has the highest overall lift values. However, the weakness of the top 5 attributes mentioned in the Edmund forum is performance. Therefore, we recommend Mercedes Benz to improve upon their sourcing or engineering of their internal components. 

#### Advertising Manager

Mercedes is known for being luxurious, safe, and spacious in the Edmunds forums. We recommend Mercedes to continue their campaigns. Mercedes lift values in luxury, safety, and space. However, we recommend that Mercedes highlight their top of the line safety features, while still advertising about the luxurious and spacious Mercedes Benz vehicles. This is due to consumers associating Mercedes Benz with Safety than with the next brand by .15, which is the most substantial difference between the 1st and 2nd place brand for all of the top 5 attributes.

### Acura

#### Product Manager

Acura is the second-highest performing brand out of the top 5 brands in the Edmunds forum. However, we recommend Acura to work on their components and engineering of their vehicles, because consumers view their performance as the least satisfactory part of Acura out of the lift values involving luxury, space, influence, price, performance, and style. Understandably, this is the most challenging category to improve, because ca manufacturing is a low margin business. 

#### Marketing Manager

Overall, Acura is the second-highest lift values for all categories. So, we recommend that Acura align its marketing with its primary target market desires. We recommend to make a better imitation of the Mercedes Benz advertising campaign, so you all can obtain their customers. This is due to Mercedes Benz having very similar lift values as Acura in “Luxury,” ”Safety,” and “Space.” 

### Audi

#### Product Manager


Unfortunately, consumers don’t associate Audi with affordable vehicles. Further, the lift value of “Audi + Price” has the lowest association out of all of the brand associations with features. However, the attribute with the least path of resistance to change is the perception is “Luxury” because we believe it will be less costly to increase the association of luxury versus price. 

#### Marketing Manager 

Audi has moderate associations with safety, space, luxury, and performance. These moderate associations make it difficult for an advertiser to create an ad campaign because Audi has a moderate association with all of these values. Therefore, we recommend imitating some of the Mercedes Benz advertising campaign, because they’re “lift leader” in your category.

### BMW 

#### Product Manager 

Unfortunately, BMW has the lowest overall lift scores out of all of the top 5 brands in the Edmunds Sedan Forum. Specifically, BMW has mediocre lift values in price, space, and safety. Given that BMW is associated mostly manufactures small and mid-size sedans. It’s acceptable that BMW isn’t associated with space. However, we believe BMW should improve the safety ratings in their vehicles. Since BMW is known to have fast cars; they’re more likely to get into accidents. Therefore, if you improve the safety rating in their vehicle. Consumers almost wreck or (unfortunately) wreck their BMW will rave about the safety quality of cars, and it won’t affect the perception of BMW being a premium brand.  

#### Marketing Manager 

BMW is known for getting a high-performance vehicle while being luxurious, according to the lift values. Therefore, BMW should pursue advertising options associated with luxury and performance. We don’t recommend imitating all portions of the Mercedes Benz marketing campaign. However, we believe there might be a few things BMW can imitate from Mercedes Benz. 

### Cadillac 

#### Product Manager 

We strongly recommend that Cadillac should improve its vehicle performance. Specifically, they should improve their internal components with increased R&D spending or increase spending on sourcing their products with suppliers. Understandably, this recommendation will be challenging for General Motors to implement, because Car manufacturing is a low margin industry. However, it’s evident in our lift analysis that Cadillac isn’t associated with well-performing vehicles.

#### Marketing Manager 

We recommend Cadillac to develop lots of tv promotions similar to Audi, Mercedes, and Acura, because of consumers associating Cadillac with space and luxury. Specifically, Cadillac is associated with luxury more than all of the top 5 most popular brands in the forum. Therefore, we recommend that Cadillac capitalize on advertising itself as the “American Luxury brand.” Globally, and within America, “#1 Luxury Car Brand in America.”

## Task E

### Most Aspirational Brand

In [ ]:
aspiration = {"dream":"aspiration","dreamt":"aspiration","dreamed":"aspiration","dreaming":"aspiration","wish":"aspiration","couldn't afford":"aspiration","love to buy":"aspiration","best":"aspiration","wanted":"aspiration","future car":"aspiration","want to":"aspiration","desire":"aspiration","desired":"aspiration","buy":"aspiration","purchase":"aspiration","will":"aspiration"}
df_a['Posts'].replace(aspiration, inplace=True, regex=True)
attributeList = ['aspiration']
brandList = [i[0] for i,j in nouncountList.most_common(10)]
lift_scores={}
for i in brand_attri_count(brandList, df_a, attributeList):
    for k,num in i[2]:
        lift_scores[(k[0],k[1])] = calculate_lift(k[0],k[1],num,i[1],i[0])
    
sorted_lifts = sorted(lift_scores.items(), key=lambda kv: kv[1], reverse=True)
sorted_lifts


In [ ]:
plotarr={}
for b in brandList:
    plotarr[b]=[]
for i in lift_scores:
    plotarr[i[0]].append(lift_scores[i])

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
colors = ("red", "green", "blue","black","orange")
groups = plotarr.keys()

for lift, color, group in zip(plotarr, colors, groups):
#     plt.scatter(attributeList,plotarr[lift])
#     ax.scatter(attributeList, plotarr[lift])
    ax.scatter(attributeList, plotarr[lift], alpha=0.8, c=color, edgecolors='none', label=group)

plt.xlabel('Combinations')
plt.ylabel('Lift Score')
plt.legend(loc=1)
plt.show()

### Analysis

We measured aspirational as a person expressing that they loved, wished or dreamed to buy a vehicle for certain brands within the forum. We decided to include the top 10 brands calculated in Task A, because we believed all of the brands have enough popularity to accurately determine if the brand is truly aspirational. We determined that the following words would allow us to calculate a lift values for all brands affiliated with aspiration, “dream”, ”dreamt”, ”dreamed” ,“dreaming”, ”wish”, ”couldn’t afford” ,“love to buy”, ”best”, ”wanted”, “future car”, ”want to”, ”desire”, ”desired”, ”buy”, ”purchase”, and ”will”. A higher lift value denotes people have a higher aspiration for the brand. 

Based on our lift scores from brand associations with aspirational words, Ford is the most aspirational brand. We find this slightly surprising, since Ford isn’t mentioned frequently within the Edmunds Forum compared to Mercedes Benz, Acura, Cadillac, and Audi. The largest business implication for this lift analysis is Ford has highly sought-after sedans. We find this implication surprising, because Ford is known for their Trucks in America. For example, the Ford F-150 (truck) is the highest sold vehicle in the United States. Therefore, the majority of American consumers associate Ford with Trucks and not sedans. Based on these facts, we assumed people wouldn’t care about Ford Sedans like the Ford Focus, Ford Fusion or the Ford Taurus.
	If we expand on this implication. It could mean Ford could potentially obtain competitors consumers in the Sedan market, or Ford needs to spend more money on marketing to expand the number of consumers who own a Ford Sedan or they need to make more vehicles or they can expand their sedan offerings into the luxury category. All of these suggested implications mean Ford could easily expand their consumer base in the American sedan market by either more marketing, building more vehicles or expanding their product-line offerings. Further, these implications align with our recommendations for Ford in Task B, where they should look into expanding in the entry-level luxury sedans. 
	Overall, it sounds like Ford has a healthy future if they capitalize on these opportunities in the American Sedan Market, because they’re highly sought after by the average American consumer who talks about Ford.
